In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import os
import pprint
import io

import pymongo
import pandas as pd
from IPython import display

import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.runrecord as runrecord_cliradlw
import climatools.cliradlw.pipeline as pipeline

In [3]:
client = pymongo.MongoClient('localhost', 27017, connect=False)

In [5]:
db = client.cliradnew

In [6]:
collection = db.lw

In [21]:
def make_doc(param=None):
    '''
    Make Mongodb document for a cliradlw run.
    
    Parameters
    ----------
    param: dict
        clirad-lw input parameters.
    doc: dict
        clirad-lw input (`param`) and output (flux and cooling rate)
    '''
    dir_fortran = pipeline.get_fortran_dir(param=param, 
                                           setup=setup_cliradlw)
    
    fnames = ['output_flux.dat', 'output_coolr.dat']
    
    s_outputs = []
    for fname in fnames:
        with open(os.path.join(dir_fortran, fname),
                  mode='r', encoding='utf-8') as f:
            s = f.read()
            
        s_outputs.append(s)
        
    doc = {fname.split('.')[0]: s
           for fname, s in zip(fnames, s_outputs)}
    doc['param'] = param
    return doc

In [20]:
params = runrecord_cliradlw.params()

In [24]:
docs = (make_doc(param=param) for param in params)
collection.insert_many(docs)

In [28]:
rs = collection.find({'param.molecule': {'h2o': 'atmpro',
                                    'n2o': 3.2e-7},
                 'param.atmpro': 'saw'})